# 1. Library Imports and Installing Dependencies

### 1.1 Install Dependencies

In [6]:
%pip install -q labelme bs4 tensorflow opencv-python-headless matplotlib albumentations scikit-learn

Note: you may need to restart the kernel to use updated packages.


### 1.2 Library Imports

In [1]:
import os
import time
import uuid
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import albumentations as alb
from sklearn.model_selection import train_test_split

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

2025-01-02 13:03:32.455430: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-02 13:03:32.575728: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/envs/facedet/lib/python3.9/site-packages/cv2/../../lib64:
2025-01-02 13:03:32.575749: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-01-02 13:03:32.610162: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alrea

## 2. Dataset

### 2.1 Directory Config

In [2]:
# Define character classes
CHARACTER_CLASSES = {
    "Sheldon": 0,
    "Leonard": 1,
    "Penny": 2,
    "Howard": 3,
    "Raj": 4,
    "Amy": 5,
    "Bernadette": 6
}
CHARACTER_NAMES = {v: k for k, v in CHARACTER_CLASSES.items()}  # Reverse mapping

# Define paths
IMG_DIR = "data/images"
LABELS_DIR = "data/labels"
TRAIN_IMAGES_DIR = os.path.join("data", "train", "images")
TRAIN_LABELS_DIR = os.path.join("data", "train", "labels")
VALIDATION_IMAGES_DIR = os.path.join("data", "validation", "images")
VALIDATION_LABELS_DIR = os.path.join("data", "validation", "labels")
TEST_IMAGES_DIR = os.path.join("data", "test", "images")
TEST_LABELS_DIR = os.path.join("data", "test", "labels")


# Define agumentation paths
AUG_DATA_DIR = "aug_data"
TRAIN_AUG_IMAGES_DIR = os.path.join(AUG_DATA_DIR, "train", "images")
TRAIN_AUG_LABELS_DIR = os.path.join(AUG_DATA_DIR, "train", "labels")
VALIDATION_AUG_IMAGES_DIR = os.path.join(AUG_DATA_DIR, "validation", "images")
VALIDATION_AUG_LABELS_DIR = os.path.join(AUG_DATA_DIR, "validation", "labels")
TEST_AUG_IMAGES_DIR = os.path.join(AUG_DATA_DIR, "test", "images")
TEST_AUG_LABELS_DIR = os.path.join(AUG_DATA_DIR, "test", "labels")

In [17]:
# Ensure necessary directories exist
os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(LABELS_DIR, exist_ok=True)
os.makedirs(TRAIN_IMAGES_DIR, exist_ok=True)
os.makedirs(TRAIN_LABELS_DIR, exist_ok=True)
os.makedirs(VALIDATION_IMAGES_DIR, exist_ok=True)
os.makedirs(VALIDATION_LABELS_DIR, exist_ok=True)
os.makedirs(TEST_IMAGES_DIR, exist_ok=True)
os.makedirs(TEST_LABELS_DIR, exist_ok=True)


os.makedirs(AUG_DATA_DIR, exist_ok=True)
os.makedirs(TRAIN_AUG_IMAGES_DIR, exist_ok=True)
os.makedirs(TRAIN_AUG_LABELS_DIR, exist_ok=True)
os.makedirs(VALIDATION_AUG_IMAGES_DIR, exist_ok=True)
os.makedirs(VALIDATION_AUG_LABELS_DIR, exist_ok=True)
os.makedirs(TEST_AUG_IMAGES_DIR, exist_ok=True)
os.makedirs(TEST_AUG_LABELS_DIR, exist_ok=True)

### 2.2 Setting up functions and URLs

In [ ]:
# URLs for each character
CHARACTER_URLS = {
    "Sheldon": "https://www.imdb.com/title/tt0898266/mediaindex/?relatedNames=nm1433588",
    "Leonard": "https://www.imdb.com/title/tt0898266/mediaindex/?relatedNames=nm0301959",
    "Penny": "https://www.imdb.com/title/tt0898266/mediaindex/?relatedNames=nm0192505",
    "Howard": "https://www.imdb.com/title/tt0898266/mediaindex/?relatedNames=nm0374865",
    "Raj": "https://www.imdb.com/title/tt0898266/mediaindex/?relatedNames=nm2471798",
    "Amy": "https://www.imdb.com/title/tt0898266/mediaindex/?relatedNames=nm0080524",
    "Bernadette": "https://www.imdb.com/title/tt0898266/mediaindex/?relatedNames=nm1851981"
}

In [ ]:
# Headers to simulate a browser
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}

In [ ]:
def create_base_directory(base_dir):
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

In [ ]:
def download_images(image_urls, save_path, limit=30, prefix=""):
    for i, url in enumerate(image_urls[:limit]):
        try:
            unique_filename = f"{prefix}_{uuid.uuid4()}.jpg"
            response = requests.get(url, headers=HEADERS, stream=True)
            if response.status_code == 200:
                with open(os.path.join(save_path, unique_filename), "wb") as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
            else:
                print(f"Error accessing image {i + 1}: {response.status_code}")
        except Exception as e:
            print(f"Error downloading image {i + 1}: {e}")

In [ ]:
def get_image_urls(imdb_url):
    try:
        response = requests.get(imdb_url, headers=HEADERS)
        soup = BeautifulSoup(response.content, 'html.parser')
        images = soup.find_all('img')
        image_urls = [
            img['src']
            for img in images
            if 'src' in img.attrs and 'media' in img['src']
        ]
        return image_urls
    except Exception as e:
        print(f"Error collecting URLs from {imdb_url}: {e}")
        return []

In [ ]:
def scrape_images():
    create_base_directory(IMG_DIR)
    for character, url in CHARACTER_URLS.items():
        print(f"Downloading images for {character}...")
        image_urls = get_image_urls(url)
        if not image_urls:
            print(f"No images found for {character}.")
            continue
        download_images(image_urls, IMG_DIR, prefix=character)
        print(f"{character}: Images downloaded successfully!")
        time.sleep(5)

### 2.3 Collecting Images

In [ ]:
scrape_images()

### 2.4 Annotate Images with LabelMe

In [11]:
!labelme

2024-12-31 18:11:03.619 | INFO     | labelme.config:get_config:66 - Loading config file from: C:\Users\welli\.labelmerc


## 3. Partition Unaugmented Data into Train, Validation, and Test Sets 

In [18]:
def split_data(img_dir, labels_dir, train_dir, val_dir, test_dir):
    """
    Split dataset into training, validation, and test directories for both images and labels.

    Args:
        img_dir (str): Path to the directory containing images.
        labels_dir (str): Path to the directory containing corresponding labels.
        train_dir (str): Path to the directory to store training images and labels.
        val_dir (str): Path to the directory to store validation images and labels.
        test_dir (str): Path to the directory to store test images and labels.
    """
    # List all image files
    images = [f for f in os.listdir(img_dir) if f.endswith('.jpg')]

    # Split data into training (70%), validation (15%), and test (15%)
    train_images, temp_images = train_test_split(images, test_size=0.3, random_state=42)
    val_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42)

    # Helper function to move images and their corresponding labels
    def move_files(file_list, src_img_dir, src_label_dir, dest_img_dir, dest_label_dir):
        for img in file_list:
            # Derive corresponding label filename
            label_file = img.replace('.jpg', '.json')
            label_path = os.path.join(src_label_dir, label_file)

            # Check if label exists; if not, skip moving the image
            if not os.path.exists(label_path):
                print(f"Skipping {img} as no corresponding label was found.")
                continue

            # Move image file
            img_path = os.path.join(src_img_dir, img)
            dest_img_path = os.path.join(dest_img_dir, img)
            os.makedirs(dest_img_dir, exist_ok=True)  # Ensure destination exists
            os.rename(img_path, dest_img_path)

            # Move label file
            dest_label_path = os.path.join(dest_label_dir, label_file)
            os.makedirs(dest_label_dir, exist_ok=True)  # Ensure destination exists
            os.rename(label_path, dest_label_path)

    # Move training data
    move_files(train_images, img_dir, labels_dir, train_dir, train_dir.replace('images', 'labels'))

    # Move validation data
    move_files(val_images, img_dir, labels_dir, val_dir, val_dir.replace('images', 'labels'))

    # Move test data
    move_files(test_images, img_dir, labels_dir, test_dir, test_dir.replace('images', 'labels'))


In [19]:
# Perform the split
split_data(IMG_DIR, LABELS_DIR, TRAIN_IMAGES_DIR, VALIDATION_IMAGES_DIR, TEST_IMAGES_DIR)

Skipping Raj_2da5b52a-ad4b-4951-94e1-510ea0b7844d.jpg as no corresponding label was found.
Skipping Raj_3c48c87f-ba72-4a4e-a550-c9c5c2c1560e.jpg as no corresponding label was found.
Skipping Leonard_b5b7e572-5645-48b6-971f-875fc5d8f5dd.jpg as no corresponding label was found.
Skipping Amy_1b70375d-3d08-4df5-8edd-d2da4b354d24.jpg as no corresponding label was found.
Skipping Raj_779fccea-45db-4305-afeb-8ef0ddc9ed0a.jpg as no corresponding label was found.
Skipping Sheldon_25d3b478-5854-4c54-8dc2-98588f6bd40f.jpg as no corresponding label was found.
Skipping Howard_bf54bdc9-0d25-4265-b3a2-e5b257427ce0.jpg as no corresponding label was found.
Skipping Bernadette_96e884f6-e4c5-442e-b73a-02eedd37720f.jpg as no corresponding label was found.
Skipping Sheldon_0b4bba54-e8a5-4844-ba21-4d8379498f78.jpg as no corresponding label was found.
Skipping Leonard_6ce2c3b9-4b6b-4c57-b456-e9d94924c3a7.jpg as no corresponding label was found.
Skipping Penny_8c54af65-ea20-46d7-869a-8c9948ad233f.jpg as no co

## 4. Image Augmentation process

In [3]:
# Define augmentation pipeline with bounding box handling
augmentor = alb.Compose([
    alb.RandomCrop(width=450, height=450), 
    alb.HorizontalFlip(p=0.5), 
    alb.RandomBrightnessContrast(p=0.2),
    alb.RandomGamma(p=0.2), 
    alb.RGBShift(p=0.2), 
    alb.VerticalFlip(p=0.5)], 
    bbox_params=alb.BboxParams(format='albumentations', label_fields=['class_labels']))

/opt/conda/envs/facedet/lib/python3.9/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `dict[str, any]` but got `UniformParams` with value `UniformParams(noise_type=...6, 0.0784313725490196)])` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [4]:
def test_augmentation_with_random_sample(train_images_dir, train_labels_dir):
    """
    Test augmentation pipeline using a random sample from the training dataset.
    """
    # Select a random image from the training dataset
    random_image = os.listdir(train_images_dir)[0]  # Using the first image for simplicity
    image_path = os.path.join(train_images_dir, random_image)
    label_path = os.path.join(train_labels_dir, random_image.replace('.jpg', '.json'))

    # Ensure the label file exists
    if not os.path.exists(label_path):
        print(f"Label file not found for image: {random_image}")
        return

    # Load the image
    img = cv2.imread(image_path)
    h, w, _ = img.shape

    # Load the label
    with open(label_path, 'r') as f:
        label = json.load(f)
    coords = label['shapes'][0]['points']
    bbox = [coords[0][0] / w, coords[0][1] / h, coords[1][0] / w, coords[1][1] / h]  # Normalize bbox

    augmented = augmentor(image=img, bboxes=[bbox], class_labels=['face'])

    # Visualize the augmented image and bounding box
    aug_img = augmented['image']
    aug_bbox = augmented['bboxes'][0]
    start_point = (int(aug_bbox[0] * aug_img.shape[1]), int(aug_bbox[1] * aug_img.shape[0]))
    end_point = (int(aug_bbox[2] * aug_img.shape[1]), int(aug_bbox[3] * aug_img.shape[0]))
    cv2.rectangle(aug_img, start_point, end_point, (255, 0, 0), 2)
    cv2.imshow("Augmented Image", aug_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
# Test the augmentation process with a random sample
test_augmentation_with_random_sample(TRAIN_IMAGES_DIR, TRAIN_LABELS_DIR)

: 

## 4. Viewing Dataset, and Build Image Loading Function

### 3.1 Limit GPU Memory Growth

In [13]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [14]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### 3.1 Load Image into TF Data Pipeline

In [15]:
images = tf.data.Dataset.list_files('..\\data\\face-detection\\images\\*.jpg')

In [16]:
images.as_numpy_iterator().next()

b'..\\data\\face-detection\\images\\Leonard_b015552d-40ce-4dd2-9f82-84b34756afd7.jpg'

In [17]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [18]:
images = images.map(load_image)

In [19]:
images.as_numpy_iterator().next()

array([[[ 69,  74,  70],
        [ 70,  75,  71],
        [ 71,  76,  72],
        ...,
        [211, 202, 187],
        [200, 191, 176],
        [193, 184, 169]],

       [[ 69,  74,  70],
        [ 70,  75,  71],
        [ 71,  76,  72],
        ...,
        [217, 208, 193],
        [206, 197, 182],
        [201, 192, 177]],

       [[ 69,  74,  70],
        [ 70,  75,  71],
        [ 71,  76,  72],
        ...,
        [225, 213, 197],
        [218, 206, 190],
        [214, 202, 186]],

       ...,

       [[150, 149, 154],
        [158, 157, 162],
        [157, 156, 161],
        ...,
        [ 54,  73, 106],
        [ 54,  73, 106],
        [ 54,  73, 106]],

       [[134, 133, 138],
        [164, 163, 168],
        [159, 158, 163],
        ...,
        [ 57,  73, 107],
        [ 57,  73, 107],
        [ 57,  73, 107]],

       [[132, 131, 136],
        [175, 174, 179],
        [162, 161, 166],
        ...,
        [ 58,  74, 108],
        [ 58,  74, 108],
        [ 58,  74, 108]]

In [20]:
type(images)

tensorflow.python.data.ops.dataset_ops.MapDataset

### 2.4 View Raw Images with Matplotlib

In [24]:
image_generator = images.batch(4).as_numpy_iterator()

In [25]:
plot_images = image_generator.next()

InvalidArgumentError: Cannot add tensor to the batch: number of elements does not match. Shapes are: [tensor]: [604,820,3], [batch]: [547,820,3] [Op:IteratorGetNext]

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, image in enumerate(plot_images):
    ax[idx].imshow(image) 
plt.show()